- Micro-management 를 지향하지는 않지만, 분단위 문의량과 직원 숫자를 추출하여 요약
- 직원들 상태값 정보/콜정보를 분단위로 쪼개서 시각화
- 업데이트 된 Plotly 사용하여 html로 결과물 배포

In [1]:
import pandas as pd
import numpy as np
import pyodbc
from datetime import time, date
from itertools import product
import plotly.express as px
import plotly

In [2]:
crawl_from = '2020-02-01'
crawl_to = '2020-03-17'

In [3]:
date_tmp = [i.strftime('%Y-%m-%d') for i in pd.date_range(start=crawl_from, end=crawl_to, freq='d')]
time_tmp = []
for h in range(10,21):
    for m in range(0,60):
        time_tmp.append(time(h, m))
        
'''
time_idx = pd.timedelta_range(start = '10:00:00', end='22:00:00', freq='min')

imedeltaIndex(['10:00:00', '10:01:00', '10:02:00', '10:03:00', '10:04:00',
                '10:05:00', '10:06:00', '10:07:00', '10:08:00', '10:09:00',
                ...
                '21:51:00', '21:52:00', '21:53:00', '21:54:00', '21:55:00',
                '21:56:00', '21:57:00', '21:58:00', '21:59:00', '22:00:00'],
               dtype='timedelta64[ns]', length=721, freq='T')
'''

"\ntime_idx = pd.timedelta_range(start = '10:00:00', end='22:00:00', freq='min')\n\nimedeltaIndex(['10:00:00', '10:01:00', '10:02:00', '10:03:00', '10:04:00',\n                '10:05:00', '10:06:00', '10:07:00', '10:08:00', '10:09:00',\n                ...\n                '21:51:00', '21:52:00', '21:53:00', '21:54:00', '21:55:00',\n                '21:56:00', '21:57:00', '21:58:00', '21:59:00', '22:00:00'],\n               dtype='timedelta64[ns]', length=721, freq='T')\n"

In [63]:
outcome = pd.DataFrame(product(date_tmp, time_tmp), columns = ['date', 'time'])
outcome['capacity'] = 0
outcome['volume'] = 0
outcome.head(5)

,date,time,capacity,volume
0,2020-02-01,10:00:00,0,0
1,2020-02-01,10:01:00,0,0
2,2020-02-01,10:02:00,0,0
3,2020-02-01,10:03:00,0,0
4,2020-02-01,10:04:00,0,0


In [64]:
outcome.tail(5)

,date,time,capacity,volume
30355,2020-03-17,20:55:00,0,0
30356,2020-03-17,20:56:00,0,0
30357,2020-03-17,20:57:00,0,0
30358,2020-03-17,20:58:00,0,0
30359,2020-03-17,20:59:00,0,0


In [6]:
conn = pyodbc.connect(server = 'RETKRCSC-NT8004', database = 'I3_IC', Trusted_Connection='yes', driver = '{ODBC Driver 13 for SQL Server}')
capacity = pd.read_sql_query( sql = '''
                    select
                    cast(statusdatetime as date) 'date'
                    , cast(StatusDateTime as time) 'start'
                    , cast(EndDateTime as time) 'end'
                    , userid
                    , statuskey
                    , stateduration

                    from
                    I3_IC.dbo.AgentActivityLog
                    where
                    cast(statusdatetime as date) >= '{start}'
                    and cast(statusdatetime as date) <= '{end}'
                    and statuskey not in ('gone home', 'lunch', 'break', 'invalid status', 'store support', 'stress management', 'technical issues', 'training')
                    and stateduration > 0
                    '''.format(start = crawl_from, end = crawl_to)
                 , con=conn)
capacity.date.astpye(str)
print(capacity.shape)
capacity.sample(5)

(208736, 6)


,date,start,end,userid,statuskey,stateduration
121294,2020-02-28,10:02:07,10:02:37,kylee11,acw_follow up,30
3114,2020-02-01,21:47:38,21:49:05,jakim20,available,87
77342,2020-02-18,20:52:51,20:57:10,soyou3,available,259
198015,2020-03-16,12:00:41,12:01:06,yukoh1,acw_follow up,25
52530,2020-02-14,10:20:23,10:23:31,yelee18,answering call,188


In [7]:
volume = pd.read_sql_query( sql = '''
                    select 
                        cast(initiateddate as date) as 'date'
                        , cast(initiateddate as time) as 'start'
                        , cast(TerminatedDate as time) as 'end'
                        ,CallDurationSeconds
                    from
                        calldetail_viw
                    where
                        cast(InitiatedDate as date) >= '{start}'
                        and cast(InitiatedDate as date) <= '{end}'
                        and nQueueWait > 0
                        and localuserid != '-'
                        and calldirection = 'inbound'
                        and AssignedWorkGroup not like '%callback%'
                        and AssignedWorkGroup not like '%media%'
                    '''.format(start = crawl_from, end = crawl_to)
                 , con=conn)
print(volume.shape)
volume.sample(5)

(50434, 4)


,date,start,end,CallDurationSeconds
15085,2020-02-16,12:52:57.853000,13:00:57.037000,479
41805,2020-03-11,17:28:52.570000,17:35:29.727000,397
39430,2020-03-09,20:14:52.397000,20:19:35.573000,283
34471,2020-03-04,15:54:26.257000,16:00:08.483000,342
39611,2020-03-08,16:03:50.890000,16:08:39.697000,289


In [36]:
capacity.date = capacity.date.astype(str)
volume.date = volume.date.astype(str)

- 여기서부터 확인

In [65]:
outcome.head()

,date,time,capacity,volume
0,2020-02-01,10:00:00,0,0
1,2020-02-01,10:01:00,0,0
2,2020-02-01,10:02:00,0,0
3,2020-02-01,10:03:00,0,0
4,2020-02-01,10:04:00,0,0


In [70]:
def separate_c(x):
    tmp = capacity[capacity.date==x.date]
    tmp = tmp[['start', 'end']]
    tmp = tmp[tmp.end > x.time]
    tmp = tmp[tmp.start < x.time]
    return len(tmp)

def separate_v(x):
    tmp = volume[volume.date==x.date]
    tmp = tmp[['start', 'end']]
    tmp = tmp[tmp.end > x.time]
    tmp = tmp[tmp.start < x.time]
    return len(tmp)

In [68]:
outcome['capacity'] = outcome.apply(separate_c, axis=1)
outcome['volume'] = outcome.apply(separate_v, axis=1)

In [77]:
outcome.to_csv('call volume and capa.csv', index=False)

___

In [152]:
px.line(outcome, x = 'time', y='volume', color= 'date', title = 'full-record of volume')
plotly.offline.plot(px.line(outcome, x = 'time', y='volume', color= 'date', title = 'full-record of volume'),
                   filename = 'full-record of volume.html')

'full-record of volume.html'

In [153]:
plotly.offline.plot(px.line(outcome, x = 'time', y='capacity', color= 'date', title = 'full-record of capacity'),
                   filename = 'full-record of capacity.html')
px.line(outcome, x = 'time', y='capacity', color= 'date', title = 'full-record of capacity')

In [82]:
outcome = outcome[outcome.date!='2020-02-29']
outcome = outcome[outcome.date!='2020-03-09']
outcome.sample(5)

,date,time,capacity,volume
74,2020-02-01,11:14:00,26,6
28338,2020-03-14,20:18:00,18,2
15708,2020-02-24,18:48:00,45,6
25762,2020-03-11,10:22:00,17,0
5092,2020-02-08,17:52:00,25,6


In [91]:
outcome['flag'] = [1*(i > '2020-03-01') for i in outcome.date]

In [94]:
outcome.groupby(['time', 'flag'], as_index=False).mean()

,time,flag,capacity,volume
0,10:00:00,0,17.793103,0.241379
1,10:00:00,1,15.133333,0.000000
2,10:01:00,0,30.137931,7.034483
3,10:01:00,1,15.466667,0.000000
4,10:02:00,0,33.275862,11.724138
5,10:02:00,1,15.733333,0.000000
6,10:03:00,0,34.482759,15.482759
7,10:03:00,1,15.866667,0.000000
8,10:04:00,0,35.620690,17.896552
9,10:04:00,1,15.800000,0.000000


In [95]:
px.line(outcome.groupby(['time', 'flag'], as_index=False).mean(), x = 'time', y='volume', color= 'flag')

In [110]:
tmp = outcome.groupby(['time', 'flag'], as_index=False).mean().melt(['flag', 'time',])
#tmp['flag'] = tmp.variable + '-' + str(tmp.flag)

In [119]:
tmp.flag = tmp.apply(lambda x: x.variable + '-' + str(x.flag), axis=1)

In [141]:
px.line(tmp[~((tmp.flag=='volume-1') & (tmp['time'] < time(10,30)))], x = 'time', y='value', color= 'flag')

In [143]:
plotly.offline.plot(px.line(tmp[~((tmp.flag=='volume-1') & (tmp['time'] < time(10,30)))], x = 'time', y='value', color= 'flag'),
                    filename='capacity and volume of call inquiries.html')

'capacity and volume of call inquiries.html'

In [149]:
tmp2 = outcome.groupby(['time', 'flag'], as_index=False).sum()
tmp2.vol_per_capacity = tmp2.volume/tmp2.capacity
tmp2.head()

,time,flag,capacity,volume,vol_per_capacity
0,10:00:00,0,516,7,0.013566
1,10:00:00,1,227,0,0.000000
2,10:01:00,0,874,204,0.233410
3,10:01:00,1,232,0,0.000000
4,10:02:00,0,965,340,0.352332


In [150]:
px.line(tmp2, x = 'time', y='vol_per_capacity', color= 'flag')

In [151]:
plotly.offline.plot(px.line(tmp2, x = 'time', y='vol_per_capacity', color= 'flag'),
                   filename= 'volume per capacity for call.html')

'volume per capacity for call.html'